# Red de colas de un Centro de Diagnóstico Automotriz
### Servicio de Inspección Técnica de Vehículos (ITV)

#### Para generar las variables aleatorias continuas se calcula la inversa de la función de distribución $F^{-1}$, llamada función cuantil.
#### $Pr\,(X\geq x)=p$, nos retorna un valor que es la probabilidad.
#### La función cuantil de cada una de las funciones de distribución necesarias son las siguientes:

#### Exponencial: $\phi^{-1}_{\lambda}=-\frac{1}{\lambda}ln(U)$, donde $U$ es la variable aleatoria del generador (0,1)
#### Normal:  $\phi^{-1}_{\mu,\sigma^{2}}=\mu+\sigma \sqrt{2}erf^{-1}(2U-1)$, donde $erf^{-1}(2U-1)=\sum_{k=0}^{\infty}\frac{c_k}{2k+1}(\frac{\sqrt{\pi}}{2}(2U-1))^{2k+1}$, donde $c_{k}=\sum_{m=0}^{k-1}\frac{c_{m}c_{k-1-m}}{(m+1)(2m+1)}=\{1,1,\frac{7}{6},\frac{127}{90},...\}$ que este último sería un vector de constantes.
#### En el caso de la uniforme simplemente se usa el generador ubicando los números en un rango a-b.
#### Uniforme: $U(0,1)=(b-a)U+a$, donde $U$, donde $a$ y $b$ son los parámetros $U(a, b)$

In [112]:
#Distribución uniforme
import math

def GCM(): #Generador RANDU
    n = 1000000
    x = 12345689
    U = [1.0]*n
    for i in range(n):
        x = (65539*x)%(2**31)
        U[i] = x/(2**31)
    return U[1:n]

def Exponencial(lam, u):
    return abs(-math.log(u)/float(lam))

def Normal(mu, sig, u):
    return abs(mu+sig*math.sqrt(2)*funcionErrorInversa(2*u-1))

def Uniforme(a, b, u):
    return (b-a)*u+a


def serieCk():
    t = 100
    Ck = [1.0]*t
    suma = 0
    for k in range(1, len(Ck)-1):
        suma = 0
        for m in range(0, k):
            suma = suma + (Ck[m]*Ck[k-1-m])/((m+1)*(2*m+1))
        Ck[k] = suma
    return Ck

def funcionErrorInversa(x):
    suma = 0
    Coeficientes = serieCk()
    for k in range(len(Coeficientes)):
        suma = suma + ((Coeficientes[k])/(2*k+1))*(math.sqrt(math.pi)/2*x)**(2*k+1)
    return suma

In [113]:
T = 200
tamaño = T**2
t=tsuc=Tp=0
M = 2**31
TSuctll1=TSucts1=TSucts2=TSucts3=M
Nll1=Nll2=Nll3=Ns1=Ns2=Ns3=n1=n2=n3=0
LL1=LL2=LL3=S1=S2=S3=[1.0]*tamaño
n_med_n1=n_med_n2=n_med_n3=0
iRandom = 0
#Parámetros de distribuciones
a=0 #uniforme (fijo)
b=1 #uniforme (fijo)
lam=1 #exponencial
mu_2=2 #exponencial
mu_1=1 #normal
sigma_1=1 #normal
mu_31=2 #normal
sigma_31=3 #normal
mu_32=5 #normal
sigma_32=6 #normal

    
def Llegada_cliente(tsuc):
    global n_med_n1,t,n1,Nll1,LL1,TSucts1,TSuctll1,iRandom,lam,mu_1,sigma_1
    n_med_n1=n_med_n1+n1*(tsuc-t)
    n1=n1+1
    Nll1=Nll1+1
    LL1[Nll1]=tsuc
    t=tsuc
    Y = Exponencial(lam, r[iRandom]) #Distribución Exponencial (Lambda)
    iRandom = iRandom+1
    if ((t+Y)<T): TSuctll1=t+Y
    if (n1==1):
        Y=Normal(mu_1, sigma_1, r[iRandom])#Distribución normal (Mu_1, Sigma_1)
        iRandom = iRandom+1
        TSucts1=t+Y
        
def Servicio_nodo1(tsuc):
    global n_med_n1,n1,t,Ns1,S1,n_med_n2,n2,Nll2,LL2,TSucts2,n_med_n3,n3,Nll3,LL3,TSucts1,TSucts3,iRandom,a,b,mu_2,mu_1,sigma_1,mu_31,sigma_31
    n_med_n1=n_med_n1+n1*(tsuc-t)
    n1=n1-1
    Ns1=Ns1+1
    S1[Ns1]=tsuc
    U=Uniforme(a, b, r[iRandom]) #Distribución uniforme(0,1)
    iRandom = iRandom+1
    if (U<=0.4):
        n_med_n2=n_med_n2+n2*(tsuc-t)
        n2=n2+1
        Nll2=Nll2+1
        LL2[Nll2]=tsuc
        if (n2==1):
            Z=Exponencial(mu_2, r[iRandom]) #Distribución Exponencial (Mu_2)
            iRandom = iRandom+1
            TSucts2=tsuc+Z
    else:
        n_med_n3=n_med_n3+n3*(tsuc-t)
        n3=n3+1
        Nll3=Nll3+1
        LL3[Nll3]=tsuc
        if (n3==1):
            W=Normal(mu_31, sigma_31, r[iRandom]) #Distribución Normal (Mu_31, Sigma_31)
            iRandom = iRandom+1
            TSucts3=tsuc+W
    t=tsuc
    if (n1>0):
        S=Normal(mu_1,sigma_1, r[iRandom]) #Distribución Normal (Mu_1, Sigma_1)
        iRandom = iRandom+1
        TSucts1=t+S
    
def Servicio_nodo2(tsuc):
    global n_med_n2,n2,t,Ns2,S2,TSucts2,n_med_n3,n3,Nll3,LL3,n3,TSucts3, iRandom,mu_2,mu_31,sigma_31
    n_med_n2=n_med_n2+n2*(tsuc-t)
    n2=n2-1
    Ns2=Ns2+1
    S2[Ns2]=tsuc
    if (n2>0):
        Y=Exponencial(mu_2, r[iRandom]) #Dist. Exponencial (Mu_2)
        iRandom = iRandom+1
        TSucts2=tsuc+Y
    n_med_n3=n_med_n3+n3*(tsuc-t)
    n3=n3+1
    Nll3=Nll3+1
    LL3[Nll3]=tsuc
    if (n3==1):
        W=Normal(mu_31, sigma_31, r[iRandom]) #Dist. Normal (Mu_31, Sigma_31)
        iRandom = iRandom+1
        TSucts3=tsuc+W
    t=tsuc
        
def Servicio_nodo3(tsuc):
    global n_med_n3,n3,t,Ns3,S3,TSucts3,iRandom,mu_32,sigma_32,mu_31,sigma_31
    n_med_n3=n_med_n3+n3*(tsuc-t)
    n3=n3-1
    Ns3=Ns3+1
    S3[Ns3]=tsuc
    if (n3>0):
        if (n3<5):
            R=Normal(mu_31, sigma_31, r[iRandom]) #Dist. Normal (Mu_31, Sigma_31)
            iRandom = iRandom+1
        else:
            R=Normal(mu_32, sigma_32, r[iRandom]) #Dist. Normal (Mu_32, Sigma_32)
            iRandom = iRandom+1
        TSucts3=tsuc+R
    t=tsuc
    
    
if __name__=="__main__":
    r = GCM()    
    X = Exponencial(lam, r[iRandom]) #Dist. Exponencial (Lambda)
    iRandom = iRandom+1
    if (X>T):
        Tp=t_medio_sistema=0
        n_med_n1=n_med_n2=n_med_n3=0
    else:
        Llegada_cliente(X)
        while ((TSuctll1 or TSucts1 or TSucts2 or TSucts3)!=M):
            if (min([TSuctll1,TSucts1,TSucts2,TSucts3])==TSuctll1):
                tsuc=TSuctll1
                TSuctll1=M
                Llegada_cliente(tsuc)
            if (min([TSuctll1,TSucts1,TSucts2,TSucts3])==TSucts1):
                tsuc=TSucts1
                TSucts1=M
                Servicio_nodo1(tsuc)
            if (min([TSuctll1,TSucts1,TSucts2,TSucts3])==TSucts2):
                tsuc=TSucts2
                TSucts2=M
                Servicio_nodo2(tsuc)
            if (min([TSuctll1,TSucts1,TSucts2,TSucts3])==TSucts3):
                tsuc=TSucts3
                TSucts3=M
                Servicio_nodo3(tsuc)
        Tp=max(0,t-T)
        acumulo1=acumulo2=acumulo3=0
        ind=0
        while (ind<Nll1):
            acumulo1=acumulo1+S1[ind]-LL1[ind]
            ind=ind+1
        ind=0
        while (ind<Nll2):
            acumulo2=acumulo2+S2[ind]-LL2[ind]
            ind=ind+1
        ind=0
        while (ind<Nll3):
            acumulo3=acumulo3+S3[ind]-LL3[ind]
            ind=ind+1
        #print (S1)
        #print(LL1)
        t_med_sistema=(acumulo1/Nll1)+(0.4*acumulo2/Nll2)+(acumulo3/Nll3)
        n_med_n1=n_med_n1/t
        n_med_n2=n_med_n2/t
        n_med_n3=n_med_n3/t
        print ("Tiempo medio del sistema:",t_med_sistema)
        print ("Tiempo medio del nodo 1:",n_med_n1)
        print ("Tiempo medio del nodo 2:",n_med_n2)
        print ("Tiempo medio del nodo 3:",n_med_n3)

Tiempo medio del sistema: 0.0
Tiempo medio del nodo 1: 25.14770986542219
Tiempo medio del nodo 2: 0.07994716268860151
Tiempo medio del nodo 3: 29.480268012779497
